In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


In [2]:
post='stc_weibo_train_post'
response='stc_weibo_train_response'

In [3]:
batch_size=64
epochs=100
latent_dim=256
num_samples=10000
split_word=" "

In [4]:
input_texts=[]
target_texts=[]
input_words=set()
target_words=set()
input_words.add(split_word)
target_words.add(split_word)

In [6]:
with open(post, encoding = 'utf8') as f:
    post_lines=f.readlines()
with open(response, encoding = 'utf8')  as f:
    response_lines=f.readlines()
for pl,rl in zip(post_lines[:min(num_samples,len(post_lines)-1)],response_lines[:min(num_samples,len(post_lines)-1)]):
    pl=pl.replace("\n","").split(split_word)
    rl=rl.replace("\n","").split(split_word)
    input_texts.append(pl)
    rl=['\t']+rl+['\n']
    target_texts.append(rl)
    for word in pl:
        if word not in input_words:
            input_words.add(word)
    for word in rl:
        if word not in target_words:
            target_words.add(word)
input_words=sorted(list(input_words))
target_words=sorted(list(target_words))
num_encoder_tokens=len(input_words)
num_decoder_tokens=len(target_words)
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])
print("Num of samples:",num_samples )
print("Num of unique input token:",num_encoder_tokens)
print("Num of unique ratget token:",num_decoder_tokens)
print("Max seq of input:",max_encoder_seq_length)
print("Max seq of target:",max_decoder_seq_length)

Num of samples: 10000
Num of unique input token: 19124
Num of unique ratget token: 16896
Max seq of input: 33
Max seq of target: 30


In [7]:
input_token_index=dict(
    [(word,i) for i,word in enumerate(input_words)]
)
target_token_index=dict(
    [(word,i) for i,word in enumerate(target_words)]
)
input_words=[]
target_words=[]

In [8]:
decoder_input_data=np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32'
)

decoder_target_data=np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32'
)
encoder_input_data=np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype='float32'
)



In [9]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,word in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[word]]=1.
    for t,word in enumerate(target_text):
        decoder_input_data[i,t,target_token_index[word]]=1.
        if t>0:
            decoder_target_data[i,t-1,target_token_index[word]]=1.

In [11]:
encoder_inputs=Input(shape=(None,num_encoder_tokens))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs,state_h,state_c=encoder(encoder_inputs)
encoder_states=[state_h,state_c]

In [14]:
decoder_inputs=Input(shape=(None,num_decoder_tokens))
decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense=Dense(num_decoder_tokens,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

In [ ]:
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)

model.compile(optimizer='rmsprop',loss='categorical_crossentropy')
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.2)
model.save('s2s.h5')

Train on 16000 samples, validate on 4000 samples
Epoch 1/100
16000/16000 [==============================] - 876s 55ms/step - loss: 2.5998 - val_loss: 2.5289
Epoch 2/100
16000/16000 [==============================] - 859s 54ms/step - loss: 2.4859 - val_loss: 2.4732
Epoch 3/100
16000/16000 [==============================] - 860s 54ms/step - loss: 2.4224 - val_loss: 2.4318
Epoch 4/100
16000/16000 [==============================] - 858s 54ms/step - loss: 2.3612 - val_loss: 2.3725
Epoch 5/100
16000/16000 [==============================] - 859s 54ms/step - loss: 2.3032 - val_loss: 2.3326
Epoch 6/100
16000/16000 [==============================] - 858s 54ms/step - loss: 2.2480 - val_loss: 2.2902
Epoch 7/100
16000/16000 [==============================] - 857s 54ms/step - loss: 2.1951 - val_loss: 2.2518
Epoch 8/100
16000/16000 [==============================] - 858s 54ms/step - loss: 2.1415 - val_loss: 2.2137
Epoch 9/100
16000/16000 [==============================] - 857s 54ms/step - loss: 2.091

In [ ]:
reverse_input_word_index=dict((i,char) for char,i in input_token_index.items())
reverse_target_word_index=dictict((i,char) for char in target_token_index.items())


In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)